In [2]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [3]:
X = np.load('Image_array.npy',allow_pickle=True)
y = np.load('Keypoints_array.npy',allow_pickle=True)

Creating Neural Network Architecture

In [ ]:
model = tf.keras.Sequential()

#Convolution layer 1
model.add(tf.keras.layers.Conv2D(32,(4,4),input_shape = (96,96,1),activation = 'elu',
          kernel_initializer = tf.keras.initializers.GlorotUniform(seed = 0)))
model.add(tf.keras.layers.MaxPool2D((2,2),strides=2))
model.add(tf.keras.layers.Dropout(0.1))

#Convolution Layer 2
model.add(tf.keras.layers.Conv2D(64,(3,3),activation='elu',
          kernel_initializer = tf.keras.initializers.GlorotUniform(seed=0)))
model.add(tf.keras.layers.MaxPool2D((2,2),strides=2))
model.add(tf.keras.layers.Dropout(0.2))
          

#Convolution Layer 3
model.add(tf.keras.layers.Conv2D(128,(2,2),activation = 'elu',
          kernel_initializer = tf.keras.initializers.GlorotUniform(seed=0)))
model.add(tf.keras.layers.MaxPool2D((2,2),strides=2))
model.add(tf.keras.layers.Dropout(0.3))         


#Convolution Layer 4
model.add(tf.keras.layers.Conv2D(256,(1,1),activation = 'elu',
          kernel_initializer = tf.keras.initializers.GlorotUniform(seed=0)))
model.add(tf.keras.layers.MaxPool2D((2,2),strides=2))
model.add(tf.keras.layers.Dropout(0.4))    

# Flatten
model.add(tf.keras.layers.Flatten())     

# Fully Connected 1
model.add(tf.keras.layers.Dense(1000,activation='elu'))
model.add(tf.keras.layers.Dropout(0.5))    

# Fully Connected 2
model.add(tf.keras.layers.Dense(1000,activation='elu'))
model.add(tf.keras.layers.Dropout(0.6))    

# Output
model.add(tf.keras.layers.Dense(30,activation='linear'))

Model Compilation and Summary

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0005,beta_1=0.9,beta_2=0.99,epsilon=1e-8)

model.compile(optimizer=opt, loss = 'mean_squared_error',metrics = ['mae'])
model.summary()

Splitting the data for training and Validation

In [4]:
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = 0.2,shuffle = True, random_state = 0)

# sanity check
print('Train Images: '+ str(len(X_train)))
print('Train Keypoints: '+ str(len(y_train)))
print('Validation Images: '+ str(len(X_val)))
print('Validation Keypoints: '+ str(len(y_val)))

Train Images: 5639
Train Keypoints: 5639
Validation Images: 1410
Validation Keypoints: 1410


Image Generator for Scaling and Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255., 
                                   zoom_range=0.1,
                                   rotation_range=20,
                                   #horizontal_flip=True,
                                   height_shift_range=0.1,
                                   width_shift_range = 0.2,
                                   fill_mode = 'nearest',
                                   brightness_range = [0.7,1.3])

validation_dategen = ImageDataGenerator(rescale=1./255.)

Fitting the Model

In [ ]:
#parameters

batch_size = 128
epochs = 200

history = model.fit(train_datagen.flow(X_train,y_train,batch_size=batch_size),
                    epochs=epochs,
                    steps_per_epoch=len(X_train)//batch_size, 
                    validation_data = validation_dategen.flow(X_val,y_val,batch_size = batch_size),
                    validation_steps = len(X_val)//batch_size)
                                                         


In [ ]:
epochs_ticks = range(epochs)
loss = history.history['loss']
val_loss = history.history['val_loss']

mae = history.history['mae']
val_mae = history.history['val_mae']                   

# plotting loss
plt.figure(figsize=(10,8),dpi=60)
plt.title('Epochs vs Loss')
plt.plot(epochs_ticks,loss,'b-',label = 'Train Loss')
plt.plot(epochs_ticks,val_loss,'r-',label = 'Validation Loss')
plt.legend(loc = 'best')

# plotting metrics
plt.figure(figsize=(10,8),dpi=60)
plt.title('Epochs vs MAE')
plt.plot(epochs_ticks,mae,'b-',label = 'Train MAE')
plt.plot(epochs_ticks,val_mae,'r-',label = 'Validation MAE')
plt.legend(loc = 'best')

plt.show()

Plotting the predicted keypoints

In [ ]:
prediction_val = model.predict(X_val/255.)

In [ ]:
import random
i=0
for i in range(4):
    idx = random.randint(1,len(X_val))
    image = X[idx].reshape(96,96)
    keypoint_true = y_val[idx]
    keypoint_pred = prediction_val[idx]
    plt.imshow(image,cmap = 'gray')
    for i in range(1,31,2):
        plt.plot(keypoint_true[i-1],keypoint_true[i],'r.')
        plt.plot(keypoint_pred[i-1],keypoint_pred[i],'b.')
        plt.xticks([]),plt.yticks([])
plt.show()

Saving the model

In [ ]:
model.save('Keypoint_CNN.h5')